In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from ydata_profiling import ProfileReport
import pandas as pd
import dash
from dash import dcc, html
from dash import dash_table
import json
import requests

In [ ]:
# Accesos de Internet Fijo
df_Accesos = pd.read_csv('https://postdata.gov.co/sites/default/files/datasets/data/ACCESOS_INTERNET_FIJO_2_20.csv', delimiter=';') 

In [ ]:
df_Accesos.info()
df_Accesos.head()

In [ ]:
df_Accesos['VELOCIDAD_EFECTIVA_DOWNSTREAM'] = pd.to_numeric(df_Accesos['VELOCIDAD_EFECTIVA_DOWNSTREAM'], errors='coerce')
df_Accesos['VELOCIDAD_EFECTIVA_UPSTREAM'] = pd.to_numeric(df_Accesos['VELOCIDAD_EFECTIVA_UPSTREAM'], errors='coerce')
df_Accesos.info()
df_Accesos.head()

In [ ]:
periodos_accesos = df_Accesos.groupby(['ANNO','TRIMESTRE'])['ACCESOS'].count().reset_index()
print(periodos_accesos)

In [ ]:
df_Accesos_3annos = df_Accesos[(df_Accesos['ANNO'] >= 2021)]
df_Accesos_3annos.info()

In [ ]:
profile_df_Accesos = ProfileReport(df_Accesos_3annos, title = 'Analisis df_Accesos')
profile_df_Accesos.to_notebook_iframe()

In [ ]:
df_Accesos_filtrado = df_Accesos[(df_Accesos['ANNO'] == 2023) & (df_Accesos['TRIMESTRE'] == 4)]
df_Accesos_filtrado.info()
df_Accesos_filtrado.head()

In [ ]:
total_accesos = df_Accesos_filtrado['ACCESOS'].sum()
print(total_accesos)

In [ ]:
go.Figure(go.Indicator(
    mode = 'gauge+number',
    value = total_accesos,
    title = {'text': 'Total de Accesos'},
    gauge = {'axis': {'range': [None, 10000000]},
             'steps' : [
                 {'range': [0, 2000000], 'color': 'gainsboro'},
                 {'range': [2000000, 4000000], 'color': 'lightgrey'},
                 {'range': [4000000, 6000000], 'color': 'lightgray'},
                 {'range': [6000000, 8000000], 'color': 'silver'}],
             'threshold' : {'line': {'color': 'red', 'width': 4}, 'thickness': 0.75, 'value': 9990000}}
))

In [ ]:
empresas = df_Accesos_filtrado.groupby(['EMPRESA'])['ACCESOS'].sum().sort_values(ascending=False).reset_index()
empresas.info()

In [ ]:
df_Accesos_filtrado2 = df_Accesos_filtrado.copy()
top_10_empresas = df_Accesos_filtrado2.groupby('EMPRESA')['ACCESOS'].sum().sort_values(ascending=False).reset_index()
top_10_empresas = top_10_empresas.sort_values('ACCESOS', ascending=False).head(10)
top_10_nombres = top_10_empresas['EMPRESA'].tolist()
df_Accesos_filtrado2['EMPRESA_TOP10'] = df_Accesos_filtrado2['EMPRESA'].apply(lambda x: x if x in top_10_nombres else 'OTROS')
top_10_empresas = df_Accesos_filtrado2.groupby('EMPRESA_TOP10')['ACCESOS'].sum().sort_values(ascending=True).reset_index()
top_10_empresas.rename(columns = {'EMPRESA_TOP10':'EMPRESA'}, inplace = True)
top_10_empresas.head(11)

In [ ]:
px.bar(top_10_empresas, y='EMPRESA', x='ACCESOS', orientation='h',
                                title='Accesos por Empresa (Top 10)', labels={'ACCESOS': 'Accesos'})

In [ ]:
top_10_empresas['Porcentaje'] = top_10_empresas['ACCESOS'] / total_accesos * 100
top_10_empresas.head(11)

In [ ]:
px.pie(top_10_empresas, values='Porcentaje', names='EMPRESA', title='Participación de Mercado (Top 10)')

In [ ]:
tecnologias = df_Accesos_filtrado.groupby(['TECNOLOGIA'])['ACCESOS'].sum().sort_values(ascending=False).reset_index()
tecnologias.info()
tecnologias.head()

In [ ]:
df_Accesos_filtrado3 = df_Accesos_filtrado.copy()
top_10_tecnologias = df_Accesos_filtrado3.groupby('TECNOLOGIA')['ACCESOS'].sum().sort_values(ascending=False).reset_index()
top_10_tecnologias = top_10_tecnologias.sort_values('ACCESOS', ascending=False).head(10)
top_10_nombres_tec = top_10_tecnologias['TECNOLOGIA'].tolist()
df_Accesos_filtrado3['TECNOLOGIA_TOP10'] = df_Accesos_filtrado3['TECNOLOGIA'].apply(lambda x: x if x in top_10_nombres_tec else 'Otras')
top_10_tecnologias = df_Accesos_filtrado3.groupby('TECNOLOGIA_TOP10')['ACCESOS'].sum().sort_values(ascending=False).reset_index()
top_10_tecnologias.rename(columns = {'TECNOLOGIA_TOP10':'TECNOLOGIA'}, inplace = True)
top_10_tecnologias['Porcentaje'] = top_10_tecnologias['ACCESOS'] / total_accesos * 100
top_10_tecnologias.head(11)

In [ ]:
tree_tecnologias = px.treemap(top_10_tecnologias, path=['TECNOLOGIA'], values='ACCESOS', title='Accesos por Tecnología', labels={'ACCESOS':'Numero de Accesos'})
tree_tecnologias.update_layout(margin = dict(t=50, l=25, r=25, b=25))
tree_tecnologias.update_layout(autosize=True, width=600, height=400)
tree_tecnologias.show()

In [ ]:
velocidades = df_Accesos_filtrado.groupby(['VELOCIDAD_EFECTIVA_DOWNSTREAM'])['ACCESOS'].sum().sort_values(ascending=False).reset_index()
velocidades.info()

In [ ]:
px.line(velocidades, x='VELOCIDAD_EFECTIVA_DOWNSTREAM', y='ACCESOS')

In [ ]:
df_Accesos_filtrado4 = df_Accesos_filtrado.copy()
etiquetas_velocidad = ['0-10', '10-50', '50-100', '100-500', '500-1000', '>1000']
bordes_velocidad = [0, 10, 50, 100, 500, 1000, 5000]
df_Accesos_filtrado4['VELOCIDAD'] = pd.cut(df_Accesos_filtrado4['VELOCIDAD_EFECTIVA_DOWNSTREAM'], bins=bordes_velocidad, labels=etiquetas_velocidad)
velocidades = df_Accesos_filtrado4.groupby('VELOCIDAD')['ACCESOS'].sum().reset_index()

In [ ]:
px.bar(velocidades, x='VELOCIDAD', y='ACCESOS', title='Velocidades de Descarga en Mbps')

In [ ]:
velocidad = (df_Accesos_filtrado['VELOCIDAD_EFECTIVA_DOWNSTREAM']*df_Accesos_filtrado['ACCESOS']).sum()
promedio_velocidad = velocidad/total_accesos
print(promedio_velocidad)

In [ ]:
go.Figure(go.Indicator(
    mode = 'gauge+number',
    value = promedio_velocidad,
    title = {'text': 'Promedio Velocidad de Descarga (Mbps)'},
    gauge = {'axis': {'range': [None, 1000]},
             'steps' : [
                 {'range': [0, 200], 'color': 'gainsboro'},
                 {'range': [200, 400], 'color': 'lightgrey'},
                 {'range': [400, 600], 'color': 'lightgray'},
                 {'range': [600, 800], 'color': 'silver'}],
             'threshold' : {'line': {'color': 'red', 'width': 4}, 'thickness': 0.75, 'value': 990}}
))

In [ ]:
segmentos = df_Accesos_filtrado.groupby(['SEGMENTO'])['ACCESOS'].sum().reset_index()
segmentos.head(10)

In [ ]:
px.bar(segmentos, x='SEGMENTO', y='ACCESOS', title='Distribución de Accesos por Estrato y Segmento')

In [ ]:
df_Accesos_filtrado5 = df_Accesos_filtrado.copy()
etiquetas_velocidad = ['0-10', '10-50', '50-100', '100-500', '500-100', '>1000']
bordes_velocidad = [0, 10, 50, 100, 500, 1000, 5000]
df_Accesos_filtrado5['VELOCIDAD'] = pd.cut(df_Accesos_filtrado5['VELOCIDAD_EFECTIVA_DOWNSTREAM'], bins=bordes_velocidad, labels=etiquetas_velocidad)
velocidades_segmento = df_Accesos_filtrado5.groupby(['SEGMENTO','VELOCIDAD'])['ACCESOS'].sum().reset_index()

In [ ]:
px.bar(velocidades_segmento, x='SEGMENTO', y='ACCESOS', color='VELOCIDAD', barmode='group', title='Distribución de Velocidades en Mbps por Estrato y Segmento')

In [ ]:
# Ingresos de Internet Fijo
df_Ingresos = pd.read_csv('https://postdata.gov.co/sites/default/files/datasets/data/INGRESOS_INTERNET_FIJO_17.csv', delimiter=';') 
df_Ingresos.info()

In [ ]:
periodos_ingresos = df_Ingresos.groupby(['ANNO','TRIMESTRE'])['INGRESOS'].count().reset_index()
print(periodos_accesos)

In [ ]:
profile_df_Ingresos = ProfileReport(df_Ingresos, title = 'Analisis df_Ingresos')
profile_df_Ingresos.to_notebook_iframe()

In [ ]:
df_Ingresos_filtrado = df_Ingresos[(df_Ingresos['ANNO'] == 2023) & (df_Ingresos['TRIMESTRE'] == 4)]
df_Ingresos_filtrado.info()

In [ ]:
total_ingresos = df_Ingresos_filtrado['INGRESOS'].sum()
print(total_ingresos)

In [ ]:
total_ingresos = total_ingresos/1000000000000
print(total_ingresos)

In [ ]:
go.Figure(go.Indicator(
    mode = 'gauge+number',
    value = total_ingresos,
    title = {'text': 'Total de Ingresos (Billones de Pesos)'},
    gauge = {'axis': {'range': [None, 2.5]},
             'steps' : [
                 {'range': [0, 0.5], 'color': 'gainsboro'},
                 {'range': [0.5, 1], 'color': 'lightgrey'},
                 {'range': [1, 1.5], 'color': 'lightgray'},
                 {'range': [1.5, 2], 'color': 'silver'}],
             'threshold' : {'line': {'color': 'red', 'width': 4}, 'thickness': 0.75, 'value': 2.49}}
))

In [ ]:
empresas_ingresos = df_Ingresos_filtrado.groupby(['EMPRESA'])['INGRESOS'].sum().sort_values(ascending=False).reset_index()
empresas_ingresos.info()

In [ ]:
df_Ingresos_filtrado2 = df_Ingresos_filtrado.copy()
top_10_empresas_ingresos = df_Ingresos_filtrado2.groupby('EMPRESA')['INGRESOS'].sum().sort_values(ascending=False).reset_index()
top_10_empresas_ingresos = top_10_empresas_ingresos.sort_values('INGRESOS', ascending=False).head(10)
top_10_nombres_ingresos = top_10_empresas_ingresos['EMPRESA'].tolist()
df_Ingresos_filtrado2['EMPRESA_TOP10'] = df_Ingresos_filtrado2['EMPRESA'].apply(lambda x: x if x in top_10_nombres_ingresos else 'OTROS')
top_10_empresas_ingresos = df_Ingresos_filtrado2.groupby('EMPRESA_TOP10')['INGRESOS'].sum().sort_values(ascending=True).reset_index()
top_10_empresas_ingresos.rename(columns = {'EMPRESA_TOP10':'EMPRESA'}, inplace = True)
top_10_empresas_ingresos['INGRESOS'] = top_10_empresas_ingresos['INGRESOS']/1000000000
top_10_empresas_ingresos.head(11)

In [ ]:
px.bar(top_10_empresas_ingresos, y='EMPRESA', x='INGRESOS', orientation='h', title='Ingresos por Empresa en Miles de Millones de Pesos (Top 10)', labels=[{'INGRESOS': 'Ingresos'},{'EMPRESA': 'Empresa'}])

In [ ]:
total_ingresos2 = df_Ingresos_filtrado['INGRESOS'].sum()
total_ingresos2 = total_ingresos2/1000000000
top_10_empresas_ingresos['Porcentaje'] = top_10_empresas_ingresos['INGRESOS'] / total_ingresos2 * 100
top_10_empresas_ingresos.head(11)

In [ ]:
px.pie(top_10_empresas_ingresos, values='Porcentaje', names='EMPRESA', title='Ingresos por Empresa (Top 10)')

In [ ]:
ingresos_servicio = df_Ingresos_filtrado.groupby(['SERVICIO'])['INGRESOS'].sum().sort_values(ascending=False).reset_index()
print(ingresos_servicio)

In [ ]:
ingresos_concepto = df_Ingresos_filtrado.groupby(['CONCEPTO'])['INGRESOS'].sum().sort_values(ascending=False).reset_index()
print(ingresos_concepto)

In [ ]:
AccesosEmpresa = df_Accesos_filtrado.groupby(['ID_EMPRESA','EMPRESA'])['ACCESOS'].sum().sort_values(ascending=False).reset_index()
AccesosEmpresa.info()
AccesosEmpresa.head(10)

In [ ]:
IngresosEmpresas = df_Ingresos_filtrado.groupby(['ID_EMPRESA','EMPRESA'])['INGRESOS'].sum().sort_values(ascending=False).reset_index()
IngresosEmpresas.info()
IngresosEmpresas.head(10)

In [ ]:
AccesosEmpresa = df_Accesos_filtrado.groupby(['ID_EMPRESA','EMPRESA'])['ACCESOS'].sum().sort_values(ascending=False).reset_index()
IngresosEmpresas = df_Ingresos_filtrado.groupby(['ID_EMPRESA','EMPRESA'])['INGRESOS'].sum().sort_values(ascending=False).reset_index()
arpu = pd.merge(AccesosEmpresa, IngresosEmpresas, on='ID_EMPRESA', how='inner')
arpu.rename(columns={'EMPRESA_x': 'EMPRESA'}, inplace=True)
arpu.drop(columns=['EMPRESA_y'], inplace=True)
arpu['ARPU'] = arpu['INGRESOS'] / arpu['ACCESOS']
arpu_top10_accesos = arpu[arpu['EMPRESA'].isin(top_10_nombres)]
arpu_top10_accesos = arpu_top10_accesos.sort_values(by='ARPU', ascending=True)
arpu_top10_accesos.head(11)

In [ ]:
px.bar(arpu_top10_accesos, y='EMPRESA', x='ARPU', orientation='h', title='Ingreso por Acceso (ARPU) Top 10 Empresas por Accesos')

In [ ]:
arpu_top10_ingresos = arpu[arpu['EMPRESA'].isin(top_10_nombres_ingresos)]
arpu_top10_ingresos = arpu_top10_ingresos.sort_values(by='ARPU', ascending=True)

In [ ]:
px.bar(arpu_top10_ingresos, y='EMPRESA', x='ARPU', orientation='h', title='Ingreso por Acceso (ARPU) Top 10 Empresas por Ingresos')

In [ ]:
# Quejas de Internet Fijo
df_Quejas = pd.read_csv('https://postdata.gov.co/sites/default/files/datasets/data/FT4_2_INT_FIJO_9.csv', delimiter=';') 
df_Quejas.info()
df_Quejas.head()

In [ ]:
periodos_quejas = df_Quejas.groupby(['ANNO','TRIMESTRE'])['NUMERO_QUEJAS'].count().reset_index()
print(periodos_quejas)

In [ ]:
profile_df_Quejas = ProfileReport(df_Quejas, title = 'Analisis df_Quejas')
profile_df_Quejas.to_notebook_iframe()

In [ ]:
df_Quejas_filtrado = df_Quejas[(df_Quejas['ANNO'] == 2023) & (df_Quejas['TRIMESTRE'] == 4)]
df_Quejas_filtrado.info()

In [ ]:
total_quejas = df_Quejas_filtrado['NUMERO_QUEJAS'].sum()
print(total_quejas)

In [ ]:
go.Figure(go.Indicator(
    mode = 'gauge+number',
    value = total_quejas,
    title = {'text': 'Total de Quejas'},
    gauge = {'axis': {'range': [None, 1000000]},
             'steps' : [
                 {'range': [0, 200000], 'color': 'gainsboro'},
                 {'range': [200000, 400000], 'color': 'lightgrey'},
                 {'range': [400000, 600000], 'color': 'lightgray'},
                 {'range': [600000, 800000], 'color': 'silver'}],
             'threshold' : {'line': {'color': 'red', 'width': 4}, 'thickness': 0.75, 'value': 990000}}
))

In [ ]:
df_Quejas_filtrado2 = df_Quejas_filtrado.copy()
top_10_empresas_quejas = df_Quejas_filtrado2.groupby('EMPRESA')['NUMERO_QUEJAS'].sum().sort_values(ascending=False).reset_index()
top_10_empresas_quejas = top_10_empresas_quejas.sort_values('NUMERO_QUEJAS', ascending=False).head(10)
top_10_nombres_quejas = top_10_empresas_quejas['EMPRESA'].tolist()
df_Quejas_filtrado2['EMPRESA_TOP10'] = df_Quejas_filtrado2['EMPRESA'].apply(lambda x: x if x in top_10_nombres_quejas else 'OTROS')
top_10_empresas_quejas = df_Quejas_filtrado2.groupby('EMPRESA_TOP10')['NUMERO_QUEJAS'].sum().sort_values(ascending=True).reset_index()
top_10_empresas_quejas.rename(columns = {'EMPRESA_TOP10':'EMPRESA'}, inplace = True)
top_10_empresas_quejas.head(11)

In [ ]:
px.bar(top_10_empresas_quejas, y='EMPRESA', x='NUMERO_QUEJAS', orientation='h', title='Quejas por Empresa (Top 10)', labels=[{'QUEJAS': 'Quejas'},{'EMPRESA': 'Empresa'}])

In [ ]:
top_10_empresas_quejas['Porcentaje'] = top_10_empresas_quejas['NUMERO_QUEJAS'] / total_quejas * 100

In [ ]:
px.pie(top_10_empresas_quejas, values='Porcentaje', names='EMPRESA', title='Quejas por Empresa (Top 10)')

In [ ]:
df_Quejas_filtrado2 = df_Quejas_filtrado.copy()
top_10_tipologia_quejas = df_Quejas_filtrado2.groupby(['TIPOLOGIA'])['NUMERO_QUEJAS'].sum().sort_values(ascending=False).reset_index()
top_10_tipologia_quejas = top_10_tipologia_quejas.sort_values('NUMERO_QUEJAS', ascending=False).head(10)
top_10_nombres_tipologia = top_10_tipologia_quejas['TIPOLOGIA'].tolist()
df_Quejas_filtrado2['TIPOLOGIA_TOP10'] = df_Quejas_filtrado2['TIPOLOGIA'].apply(lambda x: x if x in top_10_nombres_tipologia else 'Otros')
top_10_tipologia_quejas = df_Quejas_filtrado2.groupby('TIPOLOGIA_TOP10')['NUMERO_QUEJAS'].sum().sort_values(ascending=True).reset_index()
top_10_tipologia_quejas.rename(columns = {'TIPOLOGIA_TOP10':'TIPOLOGIA'}, inplace = True)
top_10_tipologia_quejas['Porcentaje'] = top_10_tipologia_quejas['NUMERO_QUEJAS'] / total_quejas * 100
top_10_tipologia_quejas.head(11)

In [ ]:
tree_tipologia_quejas = px.treemap(top_10_tipologia_quejas, path=['TIPOLOGIA'], values='NUMERO_QUEJAS', title='Quejas por Tipología', labels={'NUMERO_QUEJAS':'Número de Quejas'})
tree_tipologia_quejas.update_layout(margin = dict(t=50, l=25, r=25, b=25))
tree_tipologia_quejas.update_layout(autosize=True, width=600, height=400)
tree_tipologia_quejas.show()

In [ ]:
df_Quejas_filtrado3 = df_Quejas_filtrado.copy()
medio_quejas = df_Quejas_filtrado3.groupby(['MEDIO_ATENCION'])['NUMERO_QUEJAS'].sum().sort_values(ascending=False).reset_index()
medio_quejas['Porcentaje'] = medio_quejas['NUMERO_QUEJAS'] / total_quejas * 100
medio_quejas.head(11)

In [ ]:
px.pie(medio_quejas, values='Porcentaje', names='MEDIO_ATENCION', title='Medio de Atención Quejas')

In [ ]:
# Empaquetamiento de Servicios Fijo
df_Empaquetamiento = pd.read_csv('https://postdata.gov.co/sites/default/files/datasets/data/EMPAQUETAMIENTO_FIJO_2.csv', delimiter=';', low_memory=False)

In [ ]:
df_Empaquetamiento.info()
df_Empaquetamiento.head()

In [ ]:
periodos_empaquetamiento = df_Empaquetamiento.groupby(['ANNO','TRIMESTRE'])['CANTIDAD_LINEAS_ACCESOS'].count().reset_index()
print(periodos_empaquetamiento)

In [ ]:
profile_df_Empaquetamiento = ProfileReport(df_Empaquetamiento, title = 'Analisis df_Empaquetamiento')
profile_df_Empaquetamiento.to_notebook_iframe()

In [ ]:
servicios_empaquetamiento = df_Empaquetamiento.groupby(['SERVICIO_PAQUETE'])['CANTIDAD_LINEAS_ACCESOS'].count().reset_index()
servicios_empaquetamiento.head(10)

In [ ]:
df_Empaquetamiento = df_Empaquetamiento.drop(['VELOCIDAD_EFECTIVA_DOWNSTREAM', 'VELOCIDAD_EFECTIVA_UPSTREAM',
                                              'ID_TECNOLOGIA', 'TECNOLOGIA', 'ID_ESTADO', 'ESTADO',
                                              'VALOR_FACTURADO_O_COBRADO', 'OTROS_VALORES_FACTURADOS'], axis=1)
df_Empaquetamiento = df_Empaquetamiento.drop(df_Empaquetamiento[df_Empaquetamiento['SERVICIO_PAQUETE'] == 'Telefonía fija'].index)
df_Empaquetamiento = df_Empaquetamiento.drop(df_Empaquetamiento[df_Empaquetamiento['SERVICIO_PAQUETE'] == 'Televisión por suscripción'].index)
df_Empaquetamiento = df_Empaquetamiento.drop(df_Empaquetamiento[df_Empaquetamiento['SERVICIO_PAQUETE'] == 'Duo Play 3 (Telefonía fija y TV por suscripción)'].index)

In [ ]:
df_Empaquetamiento.info()
df_Empaquetamiento.head()

In [ ]:
df_Empaquetamiento_filtrado = df_Empaquetamiento[(df_Empaquetamiento['ANNO'] == 2023) & (df_Empaquetamiento['TRIMESTRE'] == 4)]
df_Empaquetamiento_filtrado.info()

In [ ]:
empaquetamiento_servicios = df_Empaquetamiento.groupby(['SERVICIO_PAQUETE'])['CANTIDAD_LINEAS_ACCESOS'].sum().sort_values(ascending=False).reset_index()
total_empaquetado = empaquetamiento_servicios['CANTIDAD_LINEAS_ACCESOS'].sum()
empaquetamiento_servicios['Porcentaje'] = empaquetamiento_servicios['CANTIDAD_LINEAS_ACCESOS'] / total_empaquetado * 100
empaquetamiento_servicios.head(10)

In [ ]:
px.pie(empaquetamiento_servicios, values='Porcentaje', names='SERVICIO_PAQUETE', title='Empaquetamiento de Internet Fijo')

In [ ]:
# Penetracion de Internet Fijo
df_Penetracion = pd.read_csv('https://www.datos.gov.co/resource/4py7-br84.csv?$query=SELECT%0A%20%20%60a_o%60%2C%0A%20%20%60trimestre%60%2C%0A%20%20%60cod_departamento%60%2C%0A%20%20%60departamento%60%2C%0A%20%20%60no_accesos_fijos_a_internet%60%2C%0A%20%20%60poblaci_n_dane%60%2C%0A%20%20%60indice%60%0A', delimiter=',')

In [ ]:
df_Penetracion.info()
df_Penetracion.head()

In [ ]:
df_Penetracion.rename(columns = {'a_o':'ANNO'}, inplace = True)
df_Penetracion.rename(columns = {'trimestre':'TRIMESTRE'}, inplace = True)
df_Penetracion.rename(columns = {'departamento':'DEPARTAMENTO'}, inplace = True)
df_Penetracion.rename(columns = {'no_accesos_fijos_a_internet':'ACCESOS'}, inplace = True)
df_Penetracion.rename(columns = {'poblaci_n_dane':'POBLACION'}, inplace = True)
df_Penetracion.rename(columns = {'indice':'PENETRACION'}, inplace = True)
df_Penetracion['PENETRACION'] = df_Penetracion['PENETRACION'].str.replace(',', '.')
df_Penetracion['PENETRACION'] = pd.to_numeric(df_Penetracion['PENETRACION'], errors='coerce')
df_Penetracion['cod_departamento'] = df_Penetracion['cod_departamento'].astype(str)
df_Penetracion['cod_departamento'] = df_Penetracion['cod_departamento'].str.zfill(2)
df_Penetracion.info()
df_Penetracion.head()

In [ ]:
periodos_penetracion = df_Penetracion.groupby(['ANNO','TRIMESTRE'])['ACCESOS'].count().reset_index()
print(periodos_penetracion)

In [ ]:
df_Penetracion_Faltante = df_Penetracion[(df_Penetracion['ANNO'] == 2023) & (df_Penetracion['TRIMESTRE'] == 3)]
df_Penetracion_Faltante.loc[:, 'TRIMESTRE'] = 4
df_Penetracion = pd.concat([df_Penetracion, df_Penetracion_Faltante])
periodos_penetracion = df_Penetracion.groupby(['ANNO','TRIMESTRE'])['ACCESOS'].count().reset_index()
print(periodos_penetracion)

In [ ]:
profile_df_Penetracion = ProfileReport(df_Penetracion, title = 'Analisis df_Penetracion')
profile_df_Penetracion.to_notebook_iframe()

In [ ]:
df_Penetracion_filtrado = df_Penetracion[(df_Penetracion['ANNO'] == 2023) & (df_Penetracion['TRIMESTRE'] == 3)]
df_Penetracion_filtrado.info()

In [ ]:
departamentos_penetracion = df_Penetracion_filtrado.groupby(['cod_departamento','DEPARTAMENTO'])['PENETRACION'].count().reset_index()
departamentos_penetracion.head(40)

In [ ]:
df_Penetracion_filtrado_tabla = df_Penetracion_filtrado[['DEPARTAMENTO', 'POBLACION', 'ACCESOS', 'PENETRACION']].sort_values('PENETRACION', ascending=False)

In [ ]:
go.Figure(data=[go.Table(
    header=dict(values=list(df_Penetracion_filtrado_tabla.columns)),
    cells=dict(values=[df_Penetracion_filtrado_tabla[col] for col in df_Penetracion_filtrado_tabla.columns]))
])

In [ ]:
dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df_Penetracion_filtrado_tabla.columns],
    data=df_Penetracion_filtrado_tabla.to_dict('records'),
    sort_action='native',  # Permitir ordenación nativa
    style_table={'height': '500px', 'overflowY': 'auto'},
    style_header={'backgroundColor': 'paleturquoise', 'fontWeight': 'bold'},
    style_cell={'textAlign': 'left'}
)

In [ ]:
url_geojson = 'https://csdc.immap.org/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3AMGN_DPTO_POLITICO&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326'
response = requests.get(url_geojson)
mapadepartamentos = response.json()

In [ ]:
# Crear el mapa coroplético
fig = px.choropleth(df_Penetracion_filtrado,
                    geojson=mapadepartamentos,
                    locations='cod_departamento',
                    featureidkey='properties.DPTO_CCDGO',
                    color='PENETRACION',
                    color_continuous_scale = 'PuBu',
                    hover_name='cod_departamento',
                    hover_data={'DEPARTAMENTO': True,'POBLACION': True,'ACCESOS': True},
                    labels={'DEPARTAMENTO':'DEPARTAMENTO', 'POBLACION':'POBLACION', 'ACCESOS':'ACCESOS', 'PENETRACION':'PENETRACION'})

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title="Penetración por Departamento")
fig.show()